# Comfy Kettenregel (Autograd DIY) - univariate, skalare Funktionen

$$F(x) = f_1 \circ f_2 = f_1(f_2(x)) \Rightarrow f_1'(f_2(x)) \cdot f'_2(x)$$

$$F(x) = f_1 \circ f_2 \circ f_3 = f_1(f_2(f_3(x))) \Rightarrow f_1'(f_2(f_3(x))) \cdot f_2'(f_3
(x)) \cdot f_3'(x)$$

## Aufgabe

Ziel: Gradientenbasierte Optimierung von $f(x) = \sqrt{\frac{1}{e^{\sin(x)}}}$


### 0. Imports

In [ ]:
import math
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

### 1.0 Operationen definieren

In [ ]:
def one_div_x(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = 1 / x
    derivative = -inner_derivative / x**2

    return value, derivative


def sin(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.sin(x)
    derivative = math.cos(x) * inner_derivative

    return value, derivative


def sqrt(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.sqrt(x)
    derivative = 1 / (2 * math.sqrt(x)) * inner_derivative

    return value, derivative


def exp(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.exp(x)
    derivative = math.exp(x) * inner_derivative

    return value, derivative

### 1.1 Funktionsdefinition

In [ ]:
def f_x(x: float) -> tuple[float, float]:

    return sqrt(*one_div_x(*exp(*sin(x))))

### 2. Gradient Descent

In [ ]:
x_start = 4.0  # starting value
x_min = x_start - 8.0  # x-axis limits
x_max = x_start + 8.0
xs = []  # values for the animation
ys = []

lr = 1e-2  # step size
significant_gradient = 1e-3  # termination criteria
iter = 1  # counter

while True:
    y_measured, deriv = f_x(x_start)
    if np.fabs(deriv) >= significant_gradient:
        xs.append(x_start)
        ys.append(y_measured)
        x_start -= lr * deriv
        print(iter, x_start, y_measured) if iter % 100 == 0 or iter == 1 else None
    else:
        xs.append(x_start)
        ys.append(y_measured)
        break
    iter += 1

### 3.0 Funktionsplot

In [ ]:
x = np.arange(x_min, x_max, 0.01)

res = [f_x(_) for _ in x]
y_measured, derivative = zip(*res)

df = pd.DataFrame(
    {
        "x": x,
        "y": y_measured,
        "derivative": derivative,
    }
)

px.line(df, x="x", y="y")

### 3.1 Animation

In [ ]:
# get the values
x = np.arange(x_min, x_max, 0.01)

res = [f_x(_) for _ in x]
y_measured, _ = zip(*res)

# define both graphs
fig = go.Figure(
    data=[
        go.Scatter(
            x=x,
            y=y_measured,
            mode="lines",
            line=dict(color="green", width=1),
            name="Function Graph",
        ),
        go.Scatter(
            x=[xs[0]],
            y=[ys[0]],
            mode="markers",
            marker=dict(color="red", size=10),
            name="Current Position",
        ),
    ]
)

# update layout parameters and add start button for animation
fig.update_layout(
    width=1400,
    height=900,
    xaxis=dict(range=(x_min, x_max), autorange=False),
    yaxis=dict(
        range=(np.min(y_measured) - 0.5, np.max(y_measured) + 0.5), autorange=False
    ),
    title_text="Gradient Descent Animation",
    # start button config
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    args=[
                        None,
                        {
                            "frame": {"duration": 5, "redraw": False},
                            "fromcurrent": True,
                            "transition": {"duration": 0, "easing": "linear"},
                        },
                    ],
                    label="start",
                    method="animate",
                )
            ],
        )
    ],
)

# specify the animation frames
fig.update(
    frames=[
        go.Frame(data=[go.Scatter(x=[xs[k]], y=[ys[k]])], traces=[1])
        for k in range(len(ys))
    ]
)

# show result
fig.show()

# 2024-11-18 

Bisherige Ansatz hat folgende Limitierungen
- funktioniert nur für Ausdrücke in geschlossener Form, keine Kontrollflusslogik
- inkompatibel mit binären Operatoren (+, *, ...)
- funktioniert nur in 1D

In [ ]:
import sys, os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from src import *
from src.value import *
import pandas as pd
import numpy as np
import plotly.express as px
from plotly import graph_objects as go

# Lineare Regression

In [ ]:
np.random.seed(0xDEADBEEF)

x = np.linspace(-10, 10, 200)
y_ideal = 2 * x - 2
y_measured = y_ideal + np.random.randn(len(x)) * 1.5

fig = px.scatter(x=x, y=y_measured)
fig.add_trace(go.Scatter(x=x, y=y_ideal, mode="lines"))

In [ ]:
# Lineare Regression f(x) = m*x + c
np.random.seed(0xDEADBEEF)
x = np.linspace(-10, 10, 200)
y_ideal = 2 * x - 2
y_measured = y_ideal + np.random.randn(len(x)) * 1.5

# Random init von m und c
m = Value(np.random.random(size=None) * 5, name="slope")
c = Value(np.random.random(size=None) * 5, name="intercept")


# Lossfunktion definieren
def loss(m: Value, c: Value) -> Value:
    sum_error = Value(0.0)
    for ii, x_i in enumerate(x):
        sample_error = (m * x_i + c - y_measured[ii]) ** 2
        sum_error = sum_error + sample_error
    sum_error = sum_error / len(x)
    return sum_error


# Vergleich Algorithmus mit Arithmetik
def partial_derivs(m, x, c):
    sum_dloss_dm = 0.0
    sum_dloss_dc = 0.0
    for ii, x_i in enumerate(x):
        # dloss_dm = 2 * (m * x_i + x_i * (c - y_measured[ii]))
        dloss_dm = 2 * (m * x_i + c - y_measured[ii]) * x_i
        dloss_dc = 2 * (m * x_i + c - y_measured[ii])
        sum_dloss_dm += dloss_dm
        sum_dloss_dc += dloss_dc

    return sum_dloss_dm, sum_dloss_dc


# Hyperparameter
epochs = 1000
lr = 1e-4
ms = []
cs = []
m_grad = []
c_grad = []

# Trainingloop
for i in range(epochs):

    precision_loss = loss(m, c)

    m.grad = 0
    c.grad = 0
    precision_loss.backward()

    # - Zwischenergebnisse von (m und c) speichern
    if i < 50 or i % 50 == 0:
        ms.append(m.value)
        cs.append(c.value)
        m_grad.append(m.grad)
        c_grad.append(c.grad)

    # values anhand des negativen Gradienten akkumulieren
    m.value -= lr * m.grad
    c.value -= lr * c.grad


print(f"final m: {m.value}, final c: {c.value}, final loss: {precision_loss.value}")

In [ ]:
# Vergleich analytisches Verfahren & Backwards Pass
d = partial_derivs(4.034023390966637, x, 0.9569717983633408)

print(d[0], m_grad[0], d[1], c_grad[0])

## Animation der Regression und Plot der Loss Werte

In [ ]:
ms = np.array(ms)
cs = np.array(cs)

data = []

for i, (m, c) in enumerate(zip(ms, cs)):
    ys = m * x + c
    for xi, yi in zip(x, ys):
        data.append(
            {
                "x": xi,
                "y": yi,
                "frame": i,
                "m": m,
                "c": c,
            }
        )

df = pd.DataFrame(data)


fig = px.line(df, x="x", y="y", animation_frame="frame")
fig.add_trace(go.Scatter(x=x, y=y_measured, mode="markers"))
fig.show()

In [ ]:
px.line(x=range(len(m_grad)), y=np.abs(m_grad), log_y=True)

In [ ]:
px.line(x=range(len(c_grad)), y=np.abs(c_grad), log_y=True)

# Quadratische Regression

In [ ]:
np.random.seed(0xDEADBEED)

x = np.linspace(-10, 10, 200)
y_quad_ideal = 2.0 * x**2 - 1.5 * x - 4.0
y_quad_measured = y_quad_ideal + np.random.randn(len(x)) * 20

fig = px.scatter(x=x, y=y_quad_measured)
fig.add_trace(go.Scatter(x=x, y=y_quad_ideal, mode="lines"))

In [ ]:
np.random.seed(0xDEADBEED)

a = Value(np.random.random(size=None) * 5, name="a")
b = Value(np.random.random(size=None) * 5, name="b")
c = Value(np.random.random(size=None) * 5, name="c")

x_quad = np.linspace(-10, 10, 200)
y_quad_ideal = 2.0 * x_quad**2 - 1.5 * x_quad - 4.0
y_quad_measured = y_quad_ideal + np.random.randn(len(x_quad)) * 20


# Loss Funktion
def loss_quad(x: np.ndarray, y: np.ndarray, a: Value, b: Value, c: Value) -> Value:
    sum_loss = Value(0.0)
    for x_i, y_i in zip(x, y):
        sample_loss = (a * x_i**2 + b * x_i + c - y_i) ** 2
        sum_loss = sum_loss + sample_loss
    sum_loss = sum_loss / len(x)
    sum_loss.name = "loss"
    return sum_loss


# liste/named tuple der zu optimierenden parameter
def loss_quad(x: np.ndarray, y: np.ndarray, params: list) -> Value:
    sum_loss = Value(0.0)
    for x_i, y_i in zip(x, y):
        sample_loss = (params[0] * x_i**2 + params[1] * x_i + params[2] - y_i) ** 2
        sum_loss = sum_loss + sample_loss
    sum_loss = sum_loss / len(x)
    sum_loss.name = "loss"
    return sum_loss


def partials(
    x: np.ndarray, y: np.ndarray, a: Value, b: Value, c: Value
) -> tuple[float, float, float]:
    dloss_da = 0.0
    dloss_db = 0.0
    dloss_dc = 0.0
    for ii, x_i in enumerate(x):
        dloss_da += 2 * (a * x_i**2 + b * x_i + c - y[ii]) * x_i**2
        dloss_db += 2 * (a * x_i**2 + b * x_i + c - y[ii]) * x_i
        dloss_dc += 2 * (a * x_i**2 + b * x_i + c - y[ii])

    dloss_da = dloss_da / len(x)
    dloss_db = dloss_db / len(x)
    dloss_dc = dloss_dc / len(x)

    return dloss_da, dloss_db, dloss_dc


# Learning Rate eingrenzen -> Wann e+400 Gradienten
# Ab lr von 5e-4 funktioniert Gradient descent nicht mehr
# Hyperparameter
epochs = 5000
lr = 4e-4

# Plot parameter
a_vals = []
a_grad = []
b_vals = []
b_grad = []
c_vals = []
c_grad = []
losses = []

params = [a, b, c]
# Trainingsloop
for i in range(epochs):
    loss = loss_quad(x=x_quad, y=y_quad_measured, params=params)

    for p in params:
        p.grad = 0.0

    loss.backward()
    if i < 50 or i % 50 == 0:
        a_vals.append(a.value)
        a_grad.append(a.grad)
        b_vals.append(b.value)
        b_grad.append(b.grad)
        c_vals.append(c.value)
        c_grad.append(c.grad)
        losses.append(loss.value)
    # live debugging statement
    if i % 100 == 0:
        print(f"{i}: loss: {loss}, a: {a}, b: {b}, c: {c}")

    for p in params:
        p.value -= lr * p.grad

print(f"Final loss: {loss}, final a: {a}, final b: {b}, final c: {c}")

In [ ]:
# idealer loss Wert
approximate_loss = loss_quad(
    x_quad, y_quad_measured, [Value(2.0), Value(-1.5), Value(-4.0)]
)
approximate_loss

In [ ]:
part = partials(x, y_quad_measured, a_vals[0], b_vals[0], c_vals[0])

print(part[0], a_grad[0], part[1], b_grad[0], part[2], c_grad[0])

## Animation der quadratischen Regression

In [ ]:
a_vals = np.array(a_vals)
b_vals = np.array(b_vals)
c_vals = np.array(c_vals)

data = []

for i, (a, b, c) in enumerate(zip(a_vals, b_vals, c_vals)):
    ys = a * x**2 + b * x + c
    for xi, yi in zip(x_quad, ys):
        data.append(
            {
                "x": xi,
                "y": yi,
                "frame": i,
            }
        )

df = pd.DataFrame(data=data)

fig = px.line(df, x="x", y="y", animation_frame="frame")
fig.add_trace(go.Scatter(x=x, y=y_quad_measured, mode="markers"))
fig.show()

## Plot der Gradienten-Werte

In [ ]:
# Beträge der Gradienten plotten
px.line(x=range(len(a_grad)), y=np.abs(a_grad), log_y=True)

In [ ]:
px.line(x=range(len(b_grad)), y=np.abs(b_grad), log_y=True)

In [ ]:
px.line(x=range(len(c_grad)), y=np.abs(c_grad), log_y=True)

## Plot der Loss Werte

In [ ]:
px.line(x=range(len(losses)), y=losses)